### FOREX STRATEGY 2 SMA CROSSOVER

Time Frame 1h

Indicators : 
- SMA fast 9
- SMA slow 100

LONG: When the fast SMA crosses above the slow SMA
SHORT: When the fast SMA crosses below the slow SMA

Stop Loss: 50 Pips

Target: 100 pips or close your position when reverse signal is detected 

Imports and get Data

In [1]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf


pair = 'EURUSD=X'
timeframe = '1h'
start = '2023-01-01'
end = '2023-12-31'

df = yf.download(pair, start=start, end=end, interval=timeframe)

df.head()


/Users/phitzi/PycharmProjects/Investment and Algorithmic Trading Notebooks/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-01-02 00:00:00+00:00,1.070091,1.071123,1.069862,1.070893,1.070893,0
2023-01-02 01:00:00+00:00,1.071008,1.071237,1.070091,1.070664,1.070664,0
2023-01-02 02:00:00+00:00,1.070435,1.070435,1.069633,1.070435,1.070435,0
2023-01-02 03:00:00+00:00,1.070549,1.070893,1.070091,1.070893,1.070893,0
2023-01-02 04:00:00+00:00,1.070778,1.071123,1.069862,1.070091,1.070091,0


In [2]:
def calculate_price(current_price, pips, pip_value):
    price_change = pips * pip_value
    new_price = current_price + price_change
    return new_price

#test the function
calculate_price(1.08987, -100, 0.0001)

1.0798699999999999

Backtest

In [5]:
from backtesting import Backtest, Strategy
# Uncomment if you want to use the function of crossover instead of the "hard" way
#from backtesting.lib import crossover

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

pip_value = 0.0001

class SmaCross(Strategy):
    
    sma_fast_length = 9
    sma_slow_length = 100
    sl_pips = 50
    tp_pips = 100
    
    def init(self):
        # set up the indicators
        self.smaFast = self.I(ta.sma, pd.Series(self.data.Close), self.sma_fast_length)
        self.smaSlow = self.I(ta.sma, pd.Series(self.data.Close), self.sma_slow_length)

    def next(self):
        if (self.smaFast[-1] > self.smaSlow[-1]) and (self.smaFast[-2] < self.smaSlow[-2]): # alternate code -> if crossover(self.smaFast, self.smaSlow):
            if self.position.size == 0:
                sl = calculate_price(self.data.Close[-1], -1*self.sl_pips, pip_value)
                tp = calculate_price(self.data.Close[-1], self.tp_pips, pip_value)
                self.buy(limit=self.data.Close[-1], sl=sl, tp=tp)
            elif self.position.is_short:
                self.position.close()
        if (self.smaFast[-1] < self.smaSlow[-1]) and (self.smaFast[-2] > self.smaSlow[-2]): # alternate code -> elif crossover(self.smaSlow, self.smaFast):
            if self.position.size == 0:
                sl = calculate_price(self.data.Close[-1], self.sl_pips, pip_value)
                tp = calculate_price(self.data.Close[-1], -1*self.tp_pips, pip_value)
                self.sell(limit=self.data.Close[-1],sl=sl, tp=tp)
            elif self.position.is_long:
                self.position.close()


bt = Backtest(df, SmaCross, commission=.002,
              exclusive_orders=True)
stats = bt.run()
print(stats)

Start                     2023-01-02 00:00...
End                       2023-12-29 22:00...
Duration                    361 days 22:00:00
Exposure Time [%]                   40.534413
Equity Final [$]                  8062.510222
Equity Peak [$]                  10075.895656
Return [%]                         -19.374898
Buy & Hold Return [%]                 3.09119
Return (Ann.) [%]                  -18.838864
Volatility (Ann.) [%]                 4.23719
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -21.513655
Avg. Drawdown [%]                   -4.415227
Max. Drawdown Duration      346 days 02:00:00
Avg. Drawdown Duration       69 days 18:00:00
# Trades                                   82
Win Rate [%]                        18.292683
Best Trade [%]                       0.758977
Worst Trade [%]                     -0.675301
Avg. Trade [%]                    

Optimize

In [8]:
stats, heatmap, opt = bt.optimize(
    sma_fast_length=(5,21,3),
    sma_slow_length=(50,200,25),
    sl_pips=(10, 100, 10),
    tp_pips=(10, 100, 10),
    maximize='Return [%]',

    return_optimization=True,
    method='skopt',
    constraint= lambda param: param.sl_pips < param.tp_pips)

stats

Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

InvalidParameterError: The 'criterion' parameter of ExtraTreesRegressor must be a str among {'poisson', 'squared_error', 'absolute_error', 'friedman_mse'}. Got 'mse' instead.

In [ ]:
heatmap